# Partial flight delays analysis using Dask

In [33]:
from dask import delayed
# import dask.array as da
# dask.bags
import dask.dataframe as dd
import numpy as np
import pandas as pd

### Brief experimentation with RohanApp API

In [15]:
import requests

In [16]:
url = 'https://rohan-app.herokuapp.com/api/users/1'

In [24]:
data = requests.get(url).content

In [25]:
data

b'{\n  "_links": {\n    "avatar": "https://www.gravatar.com/avatar/f7dc6dbdb953003fbe0c31a06aed0025?d=identicon&s=128", \n    "followed": "/api/users/1/followed", \n    "followers": "/api/users/1/followers", \n    "self": "/api/users/1"\n  }, \n  "about_me": "Hi.  I\'m the creator of this site.", \n  "followed_count": 2, \n  "follower_count": 2, \n  "id": 1, \n  "last_seen": "2018-05-30T06:52:34.230154Z", \n  "post_count": 25, \n  "username": "rohan"\n}\n'

### End of tangent

In [27]:
# Define @delayed-function read_flights
@delayed
def read_flights(filename):

    # Read in the DataFrame
    df = pd.read_csv(filename, parse_dates=['FL_DATE']) # FL_DATE column as datetimes

    # Calculate df['WEATHER_DELAY']
    df['WEATHER_DELAY'] = df['WEATHER_DELAY'].replace(0, np.nan)
    # Condition on there being a delay, helpful for counting the number of delays

    # Return df
    return df

In [29]:
ls flightdelays/

flightdelays-2016-1.csv  flightdelays-2016-3.csv  flightdelays-2016-5.csv
flightdelays-2016-2.csv  flightdelays-2016-4.csv


In [30]:
# Data from Bureau of Transportation Statistics
filenames = ['flightdelays-2016-1.csv', 'flightdelays-2016-2.csv',
             'flightdelays-2016-3.csv', 'flightdelays-2016-4.csv',
             'flightdelays-2016-5.csv']

In [34]:
dataframes = []

# Loop over filenames with index filename
for filename in filenames:
    # Apply read_flights to filename; append to dataframes
    dataframes.append(read_flights('flightdelays/' + filename))

# From 5 pandas dataframes to a single dask dataframe***
flight_delays = dd.from_delayed(dataframes)

# Print average of 'WEATHER_DELAY' column of flight_delays
%time print(flight_delays['WEATHER_DELAY'].mean().compute())

51.29467680608365
CPU times: user 1.59 s, sys: 239 ms, total: 1.83 s
Wall time: 1.05 s


Assuming flight was delayed (during first five months of 2016), average delay was about 51 minutes.

In [35]:
persisted_flight_delays = flight_delays.persist() # Can do this because flight_delays fits in memory!

In [36]:
%time print(persisted_flight_delays['WEATHER_DELAY'].mean().compute())

51.29467680608365
CPU times: user 11.3 ms, sys: 2.24 ms, total: 13.5 ms
Wall time: 10.1 ms


In [59]:
# Define @delayed-function read_weather with input filename
@delayed
def read_weather(filename):
    # Read in filename
    df = pd.read_csv(filename, parse_dates=['Date'])

    # Clean 'PrecipitationIn' since this column is initially object not float
    df['PrecipitationIn'] = pd.to_numeric(df['PrecipitationIn'], errors='coerce')

    # Create the 'Airport' column
    # df['Airport'] = filename.split('.')[0]
    df['Airport'] = filename[12:15]

    # Return df
    return df

In [60]:
ls weatherdata/

ATL.csv       DFW.csv       ORD.csv       pitt2016.csv
DEN.csv       MCO.csv       ord2016.csv


In [61]:
# Data from Weather Underground
filenames = ['ATL.csv', 'DFW.csv', 'ORD.csv', 'DEN.csv', 'MCO.csv']
# Atlanta, Dallas-Fort Worth, Chicago O'Hare, Denver, Orlando

In [62]:
weather_dfs = []

# Loop over filenames with filename
for filename in filenames:
    # Invoke read_weather on filename; append result to weather_dfs
    weather_dfs.append(read_weather('weatherdata/' + filename))

# Call dd.from_delayed() with weather_dfs
weather = dd.from_delayed(weather_dfs)

# Print result of weather.nlargest(1, 'Max TemperatureF')
# Prints whole row
print(weather.nlargest(1, 'Max TemperatureF').compute())

          Date  Max TemperatureF  Mean TemperatureF  Min TemperatureF  \
224 2016-08-12               107                 93                79   

     Max Dew PointF  MeanDew PointF  Min DewpointF  Max Humidity  \
224              75              71             66            79   

     Mean Humidity  Min Humidity   ...     Mean VisibilityMiles  \
224             53            27   ...                        8   

     Min VisibilityMiles  Max Wind SpeedMPH  Mean Wind SpeedMPH  \
224                    0                 41                  10   

     Max Gust SpeedMPH  PrecipitationIn  CloudCover             Events  \
224               54.0             0.82           5  Rain-Thunderstorm   

     WindDirDegrees  Airport  
224             214      DFW  

[1 rows x 24 columns]


Among the five airports, max temperature in 2016 was 107 degrees F, at DFW on Aug 12.

In [63]:
weather.columns

Index(['Date', 'Max TemperatureF', 'Mean TemperatureF', 'Min TemperatureF',
       'Max Dew PointF', 'MeanDew PointF', 'Min DewpointF', 'Max Humidity',
       'Mean Humidity', 'Min Humidity', 'Max Sea Level PressureIn',
       'Mean Sea Level PressureIn', 'Min Sea Level PressureIn',
       'Max VisibilityMiles', 'Mean VisibilityMiles', 'Min VisibilityMiles',
       'Max Wind SpeedMPH', 'Mean Wind SpeedMPH', 'Max Gust SpeedMPH',
       'PrecipitationIn', 'CloudCover', 'Events', 'WindDirDegrees', 'Airport'],
      dtype='object')

In [64]:
flight_delays.columns

Index(['FL_DATE', 'UNIQUE_CARRIER', 'FL_NUM', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'CRS_DEP_TIME', 'DEP_DELAY',
       'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 22'],
      dtype='object')

In [66]:
flight_delays_abb = flight_delays[['FL_DATE','ORIGIN','WEATHER_DELAY']]

In [67]:
weather_abb = weather[['Date','Airport','Events','PrecipitationIn']]

In [68]:
# dd.merge?
weather_delays = weather_abb.merge(flight_delays_abb, left_on=['Date','Airport'], right_on=['FL_DATE','ORIGIN'], how='inner')

In [69]:
weather_delays.columns

Index(['Date', 'Airport', 'Events', 'PrecipitationIn', 'FL_DATE', 'ORIGIN',
       'WEATHER_DELAY'],
      dtype='object')

In [70]:
type(weather_delays)

dask.dataframe.core.DataFrame

In [71]:
persisted_weather_delays = weather_delays.persist()

In [72]:
# Group persisted_weather_delays by 'Events'
by_event = persisted_weather_delays.groupby('Events')

# Count by_event['WEATHER_DELAY'] column & divide by total number of delayed flights
pct_delayed = by_event['WEATHER_DELAY'].count() / persisted_weather_delays['WEATHER_DELAY'].count() * 100

# Compute & print five largest values of pct_delayed
print(pct_delayed.nlargest(5).compute())

# Calculate mean of by_event['WEATHER_DELAY'] column & return the 5 largest entries
avg_delay_time = by_event['WEATHER_DELAY'].mean().nlargest(5)

# Compute & print avg_delay_time
print(avg_delay_time.compute())

Events
Rain-Thunderstorm        43.016760
Snow                     17.557861
Fog-Snow                  7.182761
Rain                      6.464485
Fog-Rain-Thunderstorm     4.309657
Name: WEATHER_DELAY, dtype: float64
Events
Rain-Snow                77.725490
Thunderstorm             72.500000
Fog-Rain                 55.000000
Rain-Thunderstorm        47.335807
Fog-Rain-Thunderstorm    39.685185
Name: WEATHER_DELAY, dtype: float64


'Rain-Snow' caused longest delays on average, while 'Rain-Thunderstorm' accounted for a plurality of delays.